# Файлы.

## 0. Мотивация
Данные, с которыми мы работаем, так или иначе должны где-то храниться. Один из вариантов их хранения - записывать в различные файлы. Сегодня мы будем работать с .txt файлами, потому что текстовую информацию проще воспринимать. Отдельные форматы, например, JSON, считываются немного иначе, но это выходит за рамки нашего рассмотрения вопроса.

## 1. Чтение файлов

### 1.1 Два способа открыть файл

В `Python` есть встроенная функция `open`, которая позволяет удобно работать с текстовыми файлами.

Документация по ней: https://docs.python.org/3/library/functions.html#open

Рассмотрим простейший пример (файл words.txt содержит очень много англоязычных слов для спеллчекинга):

In [2]:
my_file = open("./textfiles/words.txt")
print(type(my_file))
print(len(my_file.readlines()))
my_file.close()

<class '_io.TextIOWrapper'>
235886


Функция `open` возвращает объект, который мы можем считывать различными способами.

 Сами способы обсудим чуть ниже, а пока важно отметить - **мы обязаны закрыть файл после использования**. Иначе возникнут утечки памяти, сам файл невозможно будет использовать до завершения программы и так далее.


Конечно, мы можем про это забыть. А еще в процессе может возникнуть ошибка или что-то непредвиденное. Поэтому обычно пишутся конструкции `try-except-finally`, в которых в `finally` файл непременно закрывается. Нам каждый раз лень делать таким образом, поэтому используем иной способ.

Будем использовать `менеджер контекста`. Объяснение того, как именно это работает, выходит за рамки материала данного курса, так что предлагаю просто принять "правила игры": внутри менеджера контекста мы можем свободно работать с файлом, вне его - файл закрыт. Синтаксис в примере ниже:

In [16]:
with open("./textfiles/words.txt") as my_file:
    print(my_file.read(10))

A
a
aa
aal


### 1.2 Абсолютный и относительный пути

Вы можете заметить, что именно я отправляю в функцию `open` для открытия файла. Это путь к самому файлу в виде строки (`str`).

Путь может быть одного из двух форматов - *абсолютным* (полный путь по файловой системе до файла) или *относительным* (путь до читаемого файла от файла с кодом, который исполняется)

Примеры:

In [17]:
# Относительный путь. Обратите внимание на точку в начале - она означает, что нужно смотреть в той же папке, что и файл с кодом.
# Две точки бы означали необходимость "подняться" выше по файловой системе и так далее
with open("./textfiles/words.txt") as my_file:
    print(my_file.read(10))

print("____________________________\n")

# Заметьте, что я использую raw-строку, потому что backslash \ - это спецсимвол и
# почти всегда строки с ним не работают как мы хотим, когда мы указываем путь в windows
# Именно для того, чтобы он игнорировался как спецсимвол, и делается raw-строка
with open(r"C:\MEGA\Школа\Теоретический материал к занятиям\Модуль 3\textfiles\words.txt") as my_file:
    print(my_file.read(10))

A
a
aa
aal
____________________________
A
a
aa
aal


### 1.3 Кодировки

Отлично. Давайте попробуем прочитать первые 10 строк аналогичного words.txt словаря, но с русскими словами.

In [18]:
lines_needed = 10

with open("./textfiles/russian.txt") as rus_file:
    for _ in range(lines_needed):
        print(rus_file.readline(), end="")

-РґРµ
-РєР°
-Р»РёР±Рѕ
-РЅРёР±СѓРґСЊ
-СЃ
-С‚Р°РєРё
-С‚Рѕ
Р°
Р°-РєРѕРЅС‚Рѕ
Р°-Р»СЏ


Кажется, что-то пошло не так. Дело в кодировке. `Python` использует стандартную кодировку системы. Проверим ее:

In [19]:
import locale

print(locale.getencoding())

cp1251


`cp1251` - стандартная кодировка для русскоязычной Windows 10. Как видно, её использование не позволяет нам нормально прочитать файл.

Используем параметр `encoding` для функции `open`

In [1]:
lines_needed = 10

with open("./textfiles/russian.txt") as rus_file:
    for _ in range(lines_needed):
        print(rus_file.readline(), end="")

print("____________________________")

with open("./textfiles/russian.txt", encoding="utf-8") as rus_file:
    for _ in range(lines_needed):
        print(rus_file.readline(), end="")

-РґРµ
-РєР°
-Р»РёР±Рѕ
-РЅРёР±СѓРґСЊ
-СЃ
-С‚Р°РєРё
-С‚Рѕ
Р°
Р°-РєРѕРЅС‚Рѕ
Р°-Р»СЏ
____________________________
-де
-ка
-либо
-нибудь
-с
-таки
-то
а
а-конто
а-ля


На самом деле, происходящее зависело от кодировки самого файла. Создадим еще один, но в кодировке `cp1251`:

In [4]:
lines_needed = 10

# Не указываем кодировку => используется стандартная
with open("./textfiles/russian_in_cp1251.txt") as rus_file:
    for _ in range(lines_needed):
        print(rus_file.readline(), end="")

-де
-ка
-либо
-нибудь
-с
-таки
-то
а
а-конто
а-ля


Вывод:

Чтобы информация из нашего текстового файла правильно считывалась, нужно синхронизировать кодировки файла и функции `open` любым способом: либо изначально создавать файл в нужной кодировке, либо отправить название кодировки файла в качестве параметра `encoding` функции `open`.

## 2. Методы чтения файла


### 2.1 "Постепенное" чтение



`Python` запоминает, где мы остановились, когда считывали файл частично. 

Два самых популярных способа считать частично информацию из файла:

1) `f.read(<число>)` - считать заданное количество символов

2) `f.readline()` - считать очередную строку

In [22]:
with open("./textfiles/some_text_file.txt", encoding="utf-8") as f:
    # Читаем первую строку
    print("first print: \n    ", f.readline())
    # Читаем вторую и так далее.
    print("second print: \n    ", f.readline())

    # Перемещаем "указатель" чтения на какой-либо символ (0 - начало файла)
    f.seek(0)
    print("third print (after seek(0)): \n    ", f.readline())

    # Читаем отдельные символы
    f.seek(0)
    print("read(17) (after seek(0) again): \n    ", f.read(17))

first print: 
     Не знаю, кем я кажусь миру, но сам себе я кажусь ребёнком, который, играя на морском берегу,

second print: 
     нашёл несколько камешков поглаже и раковин попестрее, чем удавалось другим,

third print (after seek(0)): 
     Не знаю, кем я кажусь миру, но сам себе я кажусь ребёнком, который, играя на морском берегу,

read(17) (after seek(0) again): 
     Не знаю, кем я ка


### 2.2 Читаем файл целиком

`read()` прочитает весь файл и вернет многострочную строку

In [23]:
with open("./textfiles/some_text_file.txt", encoding="utf-8") as f:
    print(f.read())

Не знаю, кем я кажусь миру, но сам себе я кажусь ребёнком, который, играя на морском берегу,
нашёл несколько камешков поглаже и раковин попестрее, чем удавалось другим,
в то время как неизмеримый океан истины расстилался неисследованным перед моим взором.

Исаак Ньютон


`readlines()` прочитает весь файл и вернет список из строк (`list[str]`) (каждая отдельная строка исходного файла - это элемент списка)

In [24]:
with open("./textfiles/some_text_file.txt", encoding="utf-8") as f:
    print(f.readlines())

['Не знаю, кем я кажусь миру, но сам себе я кажусь ребёнком, который, играя на морском берегу,\n', 'нашёл несколько камешков поглаже и раковин попестрее, чем удавалось другим,\n', 'в то время как неизмеримый океан истины расстилался неисследованным перед моим взором.\n', '\n', 'Исаак Ньютон']


Заметьте, символы переноса строк `\n` внутри файла читаются вместе со строками. Но при использовании `print` эти символы будут учитываться:

In [2]:
with open("./textfiles/some_text_file.txt", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        print(line, end="")

Не знаю, кем я кажусь миру, но сам себе я кажусь ребёнком, который, играя на морском берегу,
нашёл несколько камешков поглаже и раковин попестрее, чем удавалось другим,
в то время как неизмеримый океан истины расстилался неисследованным перед моим взором.

Исаак Ньютон

## 3. Режимы открытия файлов



Очевидно, что файлы можно не только считывать, но и записывать в них что-то. Но для этого надо при вызове `open` указать специальный параметр - режим открытия файла (`mode`). По умолчанию мы открываем файлы в режиме чтения.

Все возможные режимы открытия файлов перечислены в описании функции [`open`](https://docs.python.org/3/library/functions.html#open)

In [3]:
with open("./textfiles/testfile.txt", mode="r", encoding="utf-8") as f_in:
    print("Первый раз читаем файл: \n")
    prev_text = f_in.read()
    print(prev_text)

text = """LKGBHFKLJGHBLKJDFHBGLJKDBFGK

LJBDFLKJGHBLJDRBGJIBGRTUIOYG*O#$

G*O$#*B$#@UYVB$U@#\n\n\n\n\n\nkrehbtguioervbtu83b487berubygfer"""

with open("./textfiles/testfile.txt", mode="w", encoding="utf-8") as f_out:
    f_out.write(text)

with open("./textfiles/testfile.txt", mode="r", encoding="utf-8") as f_in:
    print("\n\nВторой раз читаем файл: \n")
    print(f_in.read())

Первый раз читаем файл: 

Если что-то не сделано до конца - оно не сделано вообще.
Карл Фридрих Гаусс


Второй раз читаем файл: 

LKGBHFKLJGHBLKJDFHBGLJKDBFGK

LJBDFLKJGHBLJDRBGJIBGRTUIOYG*O#$

G*O$#*B$#@UYVB$U@#





krehbtguioervbtu83b487berubygfer


Как видим, после нашей записи, исходный текст файла стёрся. Это свойство режима `'w'` или `'w+'`.

Чтобы этого избежать, используем `'a'` для чистой дозаписи или `'r+'` для чтения и дозаписи:

In [6]:
with open("./textfiles/testfile.txt", mode="r", encoding="utf-8") as f_in:
    print("Первый раз читаем файл: \n")
    prev_text = f_in.read()
    print(prev_text)

text = """этот
текст


я

хочу добавить в свой файл"""

with open("./textfiles/testfile.txt", mode="a", encoding="utf-8") as f_out:
    f_out.write(text)

with open("./textfiles/testfile.txt", mode="r", encoding="utf-8") as f_in:
    print("\n\nВторой раз читаем файл: \n")
    print(f_in.read())

Первый раз читаем файл: 

От любой эпохи, от любой формы правления в истории остается только культурный слой.
Не остается следов от политических интриг, экономических неурядиц, смут, злодейств, войн...
Остаются пирамиды, храмы, живописные полотна, литературные произведения, музыка...
Мы читаем книги, слушаем музыку, смотрим живопись и составляем представление об эпохе.
Ведь кто бы что ныне ни говорил с больших трибун, а кроме культурного слоя, от нас ничего не останется.

Алексей Рыбников


Второй раз читаем файл: 

От любой эпохи, от любой формы правления в истории остается только культурный слой.
Не остается следов от политических интриг, экономических неурядиц, смут, злодейств, войн...
Остаются пирамиды, храмы, живописные полотна, литературные произведения, музыка...
Мы читаем книги, слушаем музыку, смотрим живопись и составляем представление об эпохе.
Ведь кто бы что ныне ни говорил с больших трибун, а кроме культурного слоя, от нас ничего не останется.

Алексей Рыбниковэтот
текст
